In [ ]:
# 필요 Library install
"""
이 셀을 실행 한 후 다음의 에러 팝업이 뜰 수 있습니다.
정상적인 것이니 Restart session 버튼을 누르고 다음 셀로 진행하시면 됩니다.

WARNING: The following packages were previously imported in this runtime:
  [_distutils_hack]
You must restart the runtime in order to use newly installed versions.

Restarting will lose all runtime state, including local variables.
"""
!pip install transformers==4.55.2 peft==0.17.0 trl==0.21.0 bitsandbytes==0.47.0 accelerate==1.10.0 vllm==0.10.1 gradio==5.43.0 pydantic==2.11.7
!pip install ipython>=8.0 jedi>=0.19

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.4/414.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# HF token 설정
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Google Drive Import
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from vllm import LLM, SamplingParams
import gradio as gr

vllm_model = LLM(
    model = "psh3333/llama3-alpaca-tuned-and-merged",
    tokenizer = "psh3333/llama3-alpaca-tuned-and-merged",
    gpu_memory_utilization=0.85,
    enable_lora = True
)

sampling_params = SamplingParams(temperature=0.05, top_p=0.95, max_tokens=256)

INFO 12-07 16:56:11 [__init__.py:241] Automatically detected platform cuda.
INFO 12-07 16:56:20 [utils.py:326] non-default args: {'model': 'psh3333/llama3-alpaca-tuned-and-merged', 'tokenizer': 'psh3333/llama3-alpaca-tuned-and-merged', 'gpu_memory_utilization': 0.85, 'disable_log_stats': True, 'enable_lora': True}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

INFO 12-07 16:56:37 [__init__.py:711] Resolved architecture: LlamaForCausalLM
INFO 12-07 16:56:37 [__init__.py:1750] Using max model len 8192
INFO 12-07 16:56:38 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

WARNING 12-07 16:56:41 [__init__.py:2921] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 12-07 16:59:48 [llm.py:298] Supported_tasks: ['generate']


In [ ]:
from vllm.lora.request import LoRARequest

dpo_output_1_base_path = "/content/drive/MyDrive/dpo_output_1/"
dpo_output_2_base_path = "/content/drive/MyDrive/dpo_output_2/"

lora_configs = {
    "dpo_output_1_cpk_10": (1, dpo_output_1_base_path + "checkpoint-10"),
    "dpo_output_1_cpk_20": (2, dpo_output_1_base_path + "checkpoint-20"),
    "dpo_output_1_cpk_30": (3, dpo_output_1_base_path + "checkpoint-30"),
    "dpo_output_1_cpk_40": (4, dpo_output_1_base_path + "checkpoint-40"),
    "dpo_output_2_cpk_10": (5, dpo_output_2_base_path + "checkpoint-10"),
    "dpo_output_2_cpk_20": (6, dpo_output_2_base_path + "checkpoint-20"),
    "dpo_output_2_cpk_30": (7, dpo_output_2_base_path + "checkpoint-30"),
    "dpo_output_2_cpk_40": (8, dpo_output_2_base_path + "checkpoint-40")
}

def generate_text(raw_input, temperature = 0.05, top_p = 0.95, max_tokens=256, lora_mode = "default"):
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens
    )
    alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    {}

    ### Response:
    {}"""
    prompt = alpaca_prompt.format(raw_input,"")
    lora_config = {}



    if lora_mode != "default":
        lora_config["lora_request"] = LoRARequest(lora_mode, lora_configs[lora_mode][0], lora_configs[lora_mode][1])

    outputs = vllm_model.generate(
        [prompt],
        sampling_params,
        **lora_config,
    )
    return outputs[0].outputs[0].text

In [ ]:
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Prompt"),
        gr.Slider(0.0, 2.0, value=0.0, label="Temperature"),
        gr.Slider(0.0, 1.0, value=0.95, label="Top P"),
        gr.Slider(1, 1000, value=256, step=1, label="Max Tokens"),
        gr.Dropdown(
            choices=["default"] + list(lora_configs.keys()),
            label="Model Type",
            value="default"
        ),
    ],
    outputs="text",
    title="Fine-tuned PEFT Model Demo",
    description="Enter a prompt to generate text using the fine-tuned model.",
    api_name = "generate"
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5a747c28c760d1b4a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
iface.close()

Closing server running on port: 7860
